In [17]:
import os
import meshio

import ufl
import dolfinx
from mpi4py import MPI
from petsc4py import PETSc # https://www.mcs.anl.gov/petsc/petsc4py-current/docs/apiref/index.html
from slepc4py import SLEPc # https://slepc.upv.es/slepc4py-current/docs/apiref/index.html

import time
import numpy as np
import matplotlib.pyplot as plt


## Load Mesh

In [18]:
MeshGeometryFolder = os.path.join("MeshGeometry")

In [19]:
OutputModalShapesFolder = os.path.join("OutputModalShapes")

In [24]:
# top_tetra_mesh = meshio.read(os.path.join(MeshGeometryFolder, "top_tetra_mesh.xdmf"))
# side_mesh = meshio.read(os.path.join(MeshGeometryFolder, "side_mesh.xml"))
# back_mesh = meshio.read(os.path.join(MeshGeometryFolder, "back_mesh.xml"))

from gmsh_helpers import read_from_msh
mesh, cell_tags, facet_tags = read_from_msh("testMesh.msh", cell_data=True, facet_data=True, gdim=3)

IndexError: index -1 is out of bounds for axis 0 with size 0

In [23]:
# top_tetra_mesh = dolfinx.mesh.Mesh()
# with XDMFFile(os.path.join(MeshGeometryFolder, "top_tetra_mesh.xdmf")) as infile:
#     infile.read(top_tetra_mesh)

## Perform Modal Analysis

References:
https://fenics-solid-tutorial.readthedocs.io/en/latest/EigenvalueProblem/EigenvalueProblem.html,
https://fenicsproject.discourse.group/t/assemble-matrix-in-dolfinx/7547

In [10]:
def epsilon(u):
    # Strain
    return 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)

def strain2voigt(e):
    return ufl.as_vector([e[0,0],e[1,1],e[2,2],e[1,2],e[0,2],e[0,1]])

def voigt2stress(s):
    return ufl.as_tensor([[s[0], s[5], s[4]],
                         [s[5], s[1], s[3]],
                         [s[4], s[3], s[2]]])
def sigma(u, C):
    # Stress
    return voigt2stress(ufl.dot(C,strain2voigt(epsilon(u))))

In [13]:
def modalAnalysisOrthotropic(params):

    # Material Parameters
    rho = params['rho']
    E_x, E_y, E_z  = params['E_x'], params['E_y'], params['E_z']
    nu_xy, nu_xz, nu_yx, nu_yz, nu_zx, nu_zy = params['nu_xy'], params['nu_xz'], params['nu_yx'], params['nu_yz'], params['nu_zx'], params['nu_zy'], 
    G_xy, G_yz, G_zx = params['G_xy'], params['G_yz'], params['G_zx']
    
    elementOrder = params['elementOrder']
    N_eig = params['N_eig']
    
    
    # Make orthotropic stiffness matrix
    delta = (1- nu_xy*nu_yx - nu_yz*nu_zy - nu_zx*nu_xz - 2*nu_xy*nu_yz*nu_zx)/(E_x * E_y * E_z)
    
    C = np.zeros((6,6))
    C[0,0] = (1-nu_yz*nu_zy)/(E_y*E_z*delta)
    C[0,1] = (nu_yx+nu_zx*nu_yz)/(E_y*E_z*delta)
    C[0,2] = (nu_zx+nu_yx*nu_zy)/(E_y*E_z*delta)
    C[1,0] = (nu_xy+nu_xz*nu_zy)/(E_z*E_x*delta)
    C[1,1] = (1-nu_zx*nu_xz)/(E_z*E_x*delta)
    C[1,2] = (nu_zy+nu_zx*nu_xy)/(E_z*E_x*delta)
    C[2,0] = (nu_xz+nu_xy*nu_yz)/(E_x*E_y*delta)
    C[2,1] = (nu_yz+nu_xz*nu_yx)/(E_x*E_y*delta)
    C[2,2] = (1-nu_xy*nu_yx)/(E_x*E_y*delta)
    C[3,3] = 2*G_yz
    C[4,4] = 2*G_zx
    C[5,5] = 2*G_xy
    
    C = ufl.as_matrix(C)    
    
    # Define Weak Formulation
    time_start = time.process_time()

    V = dolfinx.fem.VectorFunctionSpace(top_tetra_mesh, "CG", elementOrder)
    u_tr = dolfinx.TrialFunction(V)
    u_test = dolfinx.TestFunction(V)

    a_form = ufl.inner(sigma(u_tr, C), epsilon(u_test))*ufl.dx
    m_form = rho*ufl.inner(u_tr, u_test)*ufl.dx
    
    A = dolfinx.fem.petsc.assemble_matrix(a_form)
    A.assemble()
    M = dolfinx.fem.petsc.assemble_matrix(m_form)
    M.assemble()
    

    # Use Eigensolver 
    # (https://slepc.upv.es/documentation/)
    shift = SLEPc.ST().create(MPI.COMM_WORLD)
    shift.setType('sinvert')  # spectral transform
    shift.setShift(100.0)  # spectral shift
    
    eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD)
    eigensolver.setOperators(A, M)
    eigensolver.setProblemType(2) # generalized Hermitian (GHEP)
    eigensolver.setDimensions(N_eig)
    
    eigensolver.solve()
    
    time_elapsed = (time.process_time() - time_start)
    print("Time for modal analysis: {} ".format(time_elapsed))
    
    # XDMF Example: https://fenicsproject.discourse.group/t/error-in-paraview-visualization-of-xdmf-file-with-multiple-solution-variables/6873
    result_xdmf = dolfinx.io.XDMFFile(MPI.COMM_WORLD, os.path.join(OutputModalShapesFolder, "MA.xdmf"), "w")
    # result_xdmf.write_mesh(mesh)
    
    real_ev = PETSc.Vec().create(MPI.COMM_WORLD)
    comm_ev = PETSc.Vec().create(MPI.COMM_WORLD)
    
    # Get Eigenmodes
    for i in range(0, N_eig):
        # Get i-th eigenvalue and eigenvector
        eig_val = eigensolver.get_eigenpair(i, real_ev, comm_ev)
    
        #  Calculation of eigenfrequency from real part of eigenvalue
        freq_3D = np.sqrt(eig_val.real)/2/np.pi
        print("Eigenfrequency: {0:8.5f} [Hz]".format(freq_3D))
        
        
        eigenmode = dolfinx.fem.Function(V, name="Eigenvector " + str(i))
        eigenmode.vector()[:] = real_ev
        
        result_xdmf.write_function(eigenmode, i, top_mesh)
    
    xdmf.close()
    
# https://docs.fenicsproject.org/dolfinx/main/python/generated/dolfinx.io.html?highlight=xdmffile#dolfinx.io.XDMFFile




In [14]:
EnglemanSpruce = {}
EnglemanSpruce['rho'] = 435.1101985
EnglemanSpruce['E_x'] = 8.9E9
EnglemanSpruce['E_y'] = 0.128*EnglemanSpruce['E_x']*3.5
EnglemanSpruce['E_z'] = 0.059*EnglemanSpruce['E_x']
EnglemanSpruce['nu_xy'] = 0.422
EnglemanSpruce['nu_xz'] = 0.462
EnglemanSpruce['nu_yx'] = 0.083
EnglemanSpruce['nu_yz'] = 0.53
EnglemanSpruce['nu_zx'] = 0.058
EnglemanSpruce['nu_zy'] = 0.255
EnglemanSpruce['G_yz'] = 0.010*EnglemanSpruce['E_x']
EnglemanSpruce['G_zx'] = 0.120*EnglemanSpruce['E_x']
EnglemanSpruce['G_xy'] = 0.124*EnglemanSpruce['E_x']*1.5

EnglemanSpruce['elementOrder'] = 2
EnglemanSpruce['N_eig'] = 24


In [15]:
modalAnalysisOrthotropic(EnglemanSpruce)

AttributeError: 'Mesh' object has no attribute 'ufl_cell'